In [2]:
import pandas as pd
import instructor
from openai import AsyncOpenAI


df = pd.read_csv("samples/output/job_match_score.csv")

df.head()

,Unnamed: 0,Python,Pyspark,R,SQL,Hadoop,Statistical Analysis,Machine Learning,Deep Learning,GenAI,...,Support GenAI initiatives,Utilize supervised and unsupervised machine learning methodologies and NLP,Serve as a subject matter expert on Machine Learning. Deep Learning. and GenAI techniques,Collaborate with technology teams to implement analytical solutions and models,Provide additional analysis based on stakeholder needs,Python packages,location_match,industry_match,candidate_name,overall_score
0,0,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,3,5,Priya Sharma,126
1,1,5,5,0,5,5,4,5,0,2,...,1,3,2,3,4,4,3,2,Rahul Verma,86
2,2,2,1,2,4,2,3,1,0,0,...,0,0,0,2,3,1,3,2,Arjun Menon,47


In [4]:
df['candidate_name'].to_list()

['Priya Sharma', 'Rahul Verma', 'Arjun Menon']

In [5]:
from pydantic import BaseModel, Field
from enum import Enum
from typing import List, Union, Literal

class ComparisonOperator(Enum):
    """Represents comparison operators used in DataFrame filtering operations"""
    LESS_THAN = "<"
    GREATER_THAN = ">"
    EQUAL_TO = "=="
    LESSER_OR_EQUAL_TO = "<="
    GREATER_OR_EQUAL_TO = ">="


class SortOperation(BaseModel):
    """Represents a sorting operation to be applied on a DataFrame"""
    type: Literal["sort"]
    column: str = Field(description="Column name to sort the DataFrame by")
    ascending: bool = Field(description="Sort direction - True for ascending order (low to high), False for descending order (high to low)")

class FilterOperation(BaseModel):
    """Represents a filtering operation to be applied on a DataFrame"""
    type: Literal["filter"]
    column: str = Field(description="Column name to apply the filter condition on") 
    operator: ComparisonOperator = Field(description="Comparison operator (<, >, ==, <=, >=) for the filter condition")
    value: int = Field(description="Value to compare column entries against")

class DataFrameTransformation(BaseModel):
    """Represents a sequence of transformations to be applied on a DataFrame"""
    operations: List[Union[SortOperation, FilterOperation]] = Field(description="Sequence of sort and filter operations to transform the DataFrame")
    limit: int = Field(description="Maximum number of rows to return in the result")

In [11]:
column_names = df.columns.to_list()

In [8]:
example = """
Columns: ['Python', 'SQL', 'overall_score']
Query: "Show top 5 candidates with Python skills above 4"
Response:
{
    "operations": [
        {
            "type": "filter",
            "column": "Python",
            "operator": ">",
            "value": 4
        },
        {
            "type": "sort",
            "column": "Python", 
            "ascending": false
        }
    ],
    "limit": 5
}
"""

FILTER_PROMPT = """
You are an expert at converting natural language queries into structured operations for a DataFrame. Your task is to interpret a user's query and create a specification for filtering and sorting operations.
The DataFrame contains candidate resume evaluations, where each row represents a candidate and columns represent different skills and qualifications. Each column is scored on a scale of 0-5, where:
- 0 indicates no experience/skill
- 1-2 indicates basic/beginner level
- 3-4 indicates intermediate level 
- 5 indicates expert/advanced level


Given:
- A list of column names from a pandas DataFrame
- A natural language query from the user

Create a specification that:
1. Identifies the required filtering and sorting operations
2. Orders operations appropriately (filters before sorts)
3. Determines the number of rows to display
4. Returns a structured specification that can be applied to the DataFrame

For example:
{example}

The response should map to the DataFrameTransformation model with:
- operations: List containing Operation objects in execution order, where each operation has an action that is either:
  - FilterOperation:
    - column: Column name to filter on
    - operator: One of ComparisonOperator values (<, >, ==, <=, >=)
    - value: Numeric threshold value (these values range between 1-5)
  - SortOperation:
    - column: Column name to sort by
    - ascending: Sort direction (true=ascending, false=descending)
- limit: Number of rows to display in final output

**DataFrame Columns**:
{columns}

**User Query**:
{query}
"""


client = instructor.from_openai(AsyncOpenAI())


async def generate_query_filter(columns: list, user_query: str) -> DataFrameTransformation:
    prompt = FILTER_PROMPT.format(columns=columns, query=user_query, example=example)
    
    response = await client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": "You are an expert at interpreting natural language queries into structured filters."},
            {"role": "user", "content": prompt}
        ],
        response_model=DataFrameTransformation
    )
    
    return response


In [9]:
transformation = await generate_query_filter(column_names, "Show top 2 candidates with SQL experience.")

transformation

NameError: name 'column_names' is not defined

In [48]:
def apply_transformations(df: pd.DataFrame, transformation: DataFrameTransformation) -> pd.DataFrame:
    """
    Apply a series of transformations to a DataFrame based on a DataFrameTransformation object.
    
    Args:
        df: Input DataFrame
        transformation: DataFrameTransformation object containing filter and sort operations
        
    Returns:
        Transformed DataFrame with filters and sorts applied
    """
    result = df.copy()
    
    # Apply each operation in sequence
    for operation in transformation.operations:
        if isinstance(operation, FilterOperation):
            # Apply filter operation
            filter_op = operation
            if filter_op.operator == ComparisonOperator.GREATER_THAN:
                result = result[result[filter_op.column] > filter_op.value]
            elif filter_op.operator == ComparisonOperator.LESS_THAN:
                result = result[result[filter_op.column] < filter_op.value]
            elif filter_op.operator == ComparisonOperator.EQUAL_TO:
                result = result[result[filter_op.column] == filter_op.value]
            elif filter_op.operator == ComparisonOperator.GREATER_OR_EQUAL_TO:
                result = result[result[filter_op.column] >= filter_op.value]
            elif filter_op.operator == ComparisonOperator.LESSER_OR_EQUAL_TO:
                result = result[result[filter_op.column] <= filter_op.value]
                
        elif isinstance(operation, SortOperation):
            # Apply sort operation
            sort_op = operation
            result = result.sort_values(by=sort_op.column, ascending=sort_op.ascending)
    
    # Apply row limit if specified
    if transformation.limit:
        result = result.head(transformation.limit)
        
    return result


In [52]:
results_df = apply_transformations(df, transformation)

results_df.set_index("candidate_name")

,Unnamed: 0,Python,Pyspark,R,SQL,Hadoop,Statistical Analysis,Machine Learning,Deep Learning,GenAI,...,Develop predictive solutions using advanced statistical techniques,Support GenAI initiatives,Utilize supervised and unsupervised machine learning methodologies and NLP,Serve as a subject matter expert on Machine Learning. Deep Learning. and GenAI techniques,Collaborate with technology teams to implement analytical solutions and models,Provide additional analysis based on stakeholder needs,Python packages,location_match,industry_match,overall_score
candidate_name,,,,,,,,,,,,,,,,,,,,,
Priya Sharma,0,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,3,5,126
Rahul Verma,1,5,5,0,5,5,4,5,0,2,...,4,1,3,2,3,4,4,3,2,86


In [46]:
df

,Unnamed: 0,Python,Pyspark,R,SQL,Hadoop,Statistical Analysis,Machine Learning,Deep Learning,GenAI,...,Support GenAI initiatives,Utilize supervised and unsupervised machine learning methodologies and NLP,Serve as a subject matter expert on Machine Learning. Deep Learning. and GenAI techniques,Collaborate with technology teams to implement analytical solutions and models,Provide additional analysis based on stakeholder needs,Python packages,location_match,industry_match,candidate_name,overall_score
0,0,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,3,5,Priya Sharma,126
1,1,5,5,0,5,5,4,5,0,2,...,1,3,2,3,4,4,3,2,Rahul Verma,86
2,2,2,1,2,4,2,3,1,0,0,...,0,0,0,2,3,1,3,2,Arjun Menon,47


In [1]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")

In [3]:
from qdrant_client import models

type(models.Filter)

pydantic._internal._model_construction.ModelMetaclass

In [17]:
from qdrant_client.http.models import Filter, FieldCondition

In [49]:
class Transformations(BaseModel):
    filt : Filter
    limit: int = Field(description="Maximum number of records to return in the filtered result set. Use 0 or None for unlimited results.")

In [6]:
type(DataFrameTransformation)

pydantic._internal._model_construction.ModelMetaclass

In [27]:
FILTER_PROMPT = """
Given a list of columns: {columns}
And a user query: {query}

Convert this natural language query into a structured filter and limit for searching records.
The filter should use the appropriate Qdrant filter conditions and match the query intent.

Return a transformations object with:
- scroll_filter: A Qdrant Filter object that represents the filtering conditions
- limit: The maximum number of records to return (extract from query if specified)

Only include filter conditions that are explicitly mentioned or strongly implied by the query.
Use appropriate operators (must, should, range, match) based on the query semantics.

For example, if the query is "Show top 3 candidates with Python skills":
- scroll_filter would include a must condition for Python=True 
- limit would be set to 3
"""

In [58]:
client = instructor.from_openai(AsyncOpenAI())

async def generate_query_filter(columns: list, user_query: str) -> Transformations:
    prompt = FILTER_PROMPT.format(columns=columns, query=user_query)
    
    response = await client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": "You are an expert at interpreting natural language queries into structured filters."},
            {"role": "user", "content": prompt}
        ],
        strict=False,
        response_model=Filter,
        max_retries=3

    )
    
    return response


In [59]:
transformation = await generate_query_filter(column_names, "Show top 2 candidates with SQL and Python experience.")

transformation

Filter(should=None, min_should=None, must=[FieldCondition(key='SQL', match=MatchValue(value=True), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key='Python', match=MatchValue(value=True), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None)], must_not=None)

In [41]:
from qdrant_client import QdrantClient

q_client = QdrantClient(url="http://localhost:6333")

In [46]:
from app.services.file_processing import process_file
from app.services.text_extraction import extract_text

import os
from pathlib import Path

resume_folder = Path("/home/kthirumavalavan/Documents/personal/hiring_helper/samples/resumes")  # Adjust path as needed
resume_files = []

for file_path in resume_folder.glob("*.*"):
    if file_path.suffix.lower() in [".pdf", ".docx"]:
        with open(file_path, "rb") as f:
            file_content = f.read()
            file_extension = file_path.suffix.lower()[1:]  # Remove the dot
            text = extract_text(file_content, file_extension)
            resume_files.append(text)

In [47]:
resume_files

['weak-ﬁt.md 2025-02-17\n1 / 1Arjun Menon\nEmail:  arjunmenon@example.com\nPhone:  +91 76543 21098\nLinkedIn:  linkedin.com/in/arjunmenon\nLocation:  Chennai, India\nSummary\nBusiness Analyst with 6 years of experience in data analytics, SQL, and reporting . Strong background in\nworking with data-driven insights but limited exposure to advanced machine learning and GenAI .\nLooking to tr ansition into a data science role.\nSkills\nData Analysis:  SQL, Excel, Power B I, Tableau\nProgr amming:  Python (Basic), R (Basic)\nStatistical Analysis:  Regression, Descriptive Analytics\nBig Data:  Familiarity with Hadoop & Spark\nExperience\nBusiness Analyst | JKL Consulting | May 2018 – Present\nAnalyzed customer behavior trends to optimize marketing str ategies .\nCreated SQL-based data pipelines  for financial reporting.\nAssisted in basic predictive modeling  but did not directly implement ML models.\nData Analyst | MNO Corp | Aug 2016 – April 2018\nConducted ad-hoc data analysis  using SQL 

In [62]:
# Load resume files into embeddings
from qdrant_client.http import models
from fastembed import TextEmbedding

# Create collection for resumes if it doesn't exist
collection_name = "resumes"
q_client.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=384,  # FastEmbed dimension
        distance=models.Distance.COSINE
    )
)

# Convert resumes to embeddings using FastEmbed

embedding_model = TextEmbedding()

resume_embeddings = []
for text in resume_files:
    embeddings = list(embedding_model.embed([text]))
    resume_embeddings.append(embeddings[0])

# Create payloads from DataFrame rows
payloads = df[['candidate_name']].to_dict('records')

# Upload vectors with payloads to Qdrant
q_client.upsert(
    collection_name=collection_name,
    points=models.Batch(
        ids=list(range(len(resume_embeddings))),
        vectors=resume_embeddings,
        payloads=payloads
    )
)



/tmp/ipykernel_6676/1187074189.py:7: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  q_client.recreate_collection(


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
jd_embedding = None
query = ""
dense_embedding_model = TextEmbedding("sentence-transformers/all-MiniLM-L6-v2")
reranker = TextCrossEncoder(model_name='jinaai/jina-reranker-v2-base-multilingual')

query_embedded = list(dense_embedding_model.query_embed(query))[0]

initial_retrieval = q_client.query_points(
    collection_name="movies",
    using="embedding",
    query_filter=None,
    query=query_embedded,
    with_payload=True,

    limit=10
)

new_scores = list(reranker.rerank(query, description_hits)) #returns scores between query and each document

ranking = [(i, score) for i, score in enumerate(new_scores)] #saving document indices
ranking.sort(key=lambda x: x[1], reverse=True) #sorting them in order of relevance defined by reranker

for i, rank in enumerate(ranking):
    print(f'''Reranked result number {i+1} is \"{description_hits[rank[0]]}\"''')

In [61]:
df[['candidate_name']].to_dict('records')

[{'candidate_name': 'Priya Sharma'},
 {'candidate_name': 'Rahul Verma'},
 {'candidate_name': 'Arjun Menon'}]

In [67]:
points, _ = q_client.scroll(
    collection_name="resumes",
    scroll_filter=models.Filter(must=[models.FieldCondition(key="candidate_name", match=models.MatchAny(any=["Priya Sharma"]))]),
    with_vectors=True)

In [68]:
points

[Record(id=0, payload={'candidate_name': 'Priya Sharma'}, vector=[0.007914781, 0.0018578588, -0.025483044, -0.038936157, -0.029564627, -0.04919382, 0.062996015, 0.07529447, 0.0056021074, -0.0452465, -0.00012807394, -0.021213492, 0.040251933, 0.026691407, 0.03361936, -0.038184285, 0.04237328, -0.043286268, 0.029000724, 0.020756999, -0.00187632, -0.033485096, -0.06970915, -0.047233585, 0.0494892, -0.038184285, -0.033270277, -0.0820076, -0.0066728517, -0.14629254, 0.020622738, -0.0068742456, 0.07411296, -0.03289434, 0.05228186, 0.04452148, -0.046723388, 0.03566015, -0.0025140673, -0.025254797, -0.051368877, -0.04457519, -0.014137853, 0.022757513, -0.008599521, -0.031229483, 0.012875784, -0.038076878, -0.0627812, -0.036707398, -0.015601316, -0.068796165, 0.027483556, 0.047636375, 0.028517378, -0.0133389905, 0.036250904, 0.023173727, 0.011231068, 0.039580617, 0.07604634, -0.004910655, -0.123843834, 0.019280111, -0.010210671, 0.06723872, 0.003883546, -0.092157856, -0.009458801, 0.02628862, -

In [ ]:
reranker.rerank(query, points)

In [1]:
str({'k': 'v'})

"{'k': 'v'}"

In [ ]:
first step: metadata filtering

second step reranking by embedding rows of the filtered points


flowchart TD
    A[Resume CSV Upload] --> B[Process CSV File]
    B --> C[QdrantManager]
    
    subgraph QdrantManager Setup
        C --> D[Initialize Qdrant Client]
        D --> E[Create 'resumes' Collection]
        E --> F[Initialize FastEmbed & Reranker Models]
    end
    
    subgraph Upsert Process
        G[DataFrame with Resumes] --> H[Convert Each Row to Text]
        H --> I[Generate Embeddings]
        I --> J[Create Payloads with Names & Scores]
        J --> K[Upsert to Qdrant DB]
    end
    
    subgraph Search & Rerank
        L[User Query] --> M[Filter by Candidate Names]
        M --> N[Retrieve Matching Points]
        N --> O[Extract Scores]
        O --> P[Rerank using Cross-Encoder]
        P --> Q[Return Ranked Results]
    end
    
    C --> G
    L --> C
    
    style QdrantManager Setup fill:#f9f,stroke:#333
    style Upsert Process fill:#bbf,stroke:#333
    style Search & Rerank fill:#bfb,stroke:#333